In [5]:
from driver_manager import DriverManager
from employee_data_crawler import EmployeeDataCrawler
from time_off_calendar_crawler import TimeOffCalendarCrawler
from uploader import Uploader
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import time
import traceback
import os
import sys
import util

def username_password_input() -> (str, str):
    print('Client - Paycom Username:')
    username = input()
    print('Client - Paycom Password')
    password = input()
    return username, password

def terminate():
    driver.quit()

def get_session_ready() -> bool:
    username, password = username_password_input()
    login_result = driver_manager.login(username, password)
    if login_result == True:
        driver = driver_manager.driver_hand_over()
        return True
    else:
        return False

driver_manager = DriverManager()
driver = None
base_dir = ''
if getattr(sys, 'frozen', False):
    base_dir = os.path.dirname(sys.executable)
else:
    base_dir = os.path.abspath('')

is_session_ready = get_session_ready()
if is_session_ready == True:
    print("LOGIN SUCCESSFUL. SESSION IS READY.")
    driver = driver_manager.driver_hand_over()
else:
    print("ERROR. LOGIN FAILED. SESSION IS NOT READY.")
    exit()

driver.get("https://www.paycomonline.net/v4/cl/web.php/employee/changes")
delay = 10
WebDriverWait(driver, delay).until(
    expected_conditions.presence_of_element_located((By.CLASS_NAME, 'css-vkr5jj')))
print("'Make Employee Changes' page loaded")

result = []



2023-05-30 21:23:48,555 WARNING Your selenium-driver-updater library is out of date,please update it via "pip install selenium-driver-updater --upgrade" current_version: 5.1.7 latest_version: 5.1.8  


temp test


2023-05-30 21:23:48,961 INFO Current version of chromedriver: 113.0.5672.63 
2023-05-30 21:23:49,049 INFO Latest version of chromedriver: 113.0.5672.63 
2023-05-30 21:23:49,050 INFO Your existing chromedriver is up to date. current_version: 113.0.5672.63 latest_version: 113.0.5672.63 


Chrome User Data Path: C:\Users\neils\PycharmProjects\paycomCrawler\chrome_user_data
Client - Paycom Username:


 sangjunlee


Client - Paycom Password


 payNeh@85246


logged in
LOGIN SUCCESSFUL. SESSION IS READY.
'Make Employee Changes' page loaded


In [11]:
def retry_action(action, max_retries=3, delay=3):
    retries = 0
    while retries < max_retries:
        try:
            return action()
        except StaleElementReferenceException:
            retries += 1
            time.sleep(delay)
    raise StaleElementReferenceException("Max retries exceeded.")


def retry_find_element(source, by, locator, max_retries=3, delay=2):
    retries = 0
    while retries < max_retries:
        try:
            return source.find_element(by, locator)
        except StaleElementReferenceException:
            retries += 1
            time.sleep(delay)
    raise StaleElementReferenceException("Max retries exceeded.")


def retry_find_elements(source, by, locator, max_retries=3, delay=2):
    retries = 0
    while retries < max_retries:
        try:
            return source.find_elements(by, locator)
        except StaleElementReferenceException:
            retries += 1
            time.sleep(delay)
    raise StaleElementReferenceException("Max retries exceeded.")



result = []


users = retry_find_elements(driver, By.CLASS_NAME, 'css-vkr5jj')

employee_ids = []

driver.execute_script("return document.body.scrollHeight")

before_page_y_offset = driver.execute_script("return window.pageYOffset;")
not_hit_bottom = 1

while not_hit_bottom:
    
    try:
        employees = retry_find_elements(driver, By.CSS_SELECTOR, 'div.css-gwrvcm')
        one_row_height = 81

        for each_employee in employees:

            result_dict = {}

            # one_row_height = each_employee.size['height']

            employee_id = retry_find_element(each_employee, By.CSS_SELECTOR, 'span.css-20f01v').text
            if employee_id not in employee_ids:
                
                element = retry_find_element(each_employee, By.CSS_SELECTOR, 'button.css-kb77mr')
                element.click()
                # driver.implicitly_wait(10)
                time.sleep(5)
                
                
                driver.execute_script("arguments[0].scrollIntoView();", retry_find_element(driver, By.CSS_SELECTOR, 'div.css-927gzi'))
                time.sleep(5)

                ###
                # temp_name = each_employee.find_element(By.CSS_SELECTOR, 'div.css-ewvkoh').find_element(By.CSS_SELECTOR, 'h1.jssEEFormsRewrite17').text
                element = retry_find_element(each_employee, By.CSS_SELECTOR, 'div.css-ewvkoh')
                temp_name = retry_find_element(element, By.CSS_SELECTOR, 'h1.jssEEFormsRewrite17').text
                driver.implicitly_wait(10)

                result_dict['name'] = util.name_formatter(temp_name)

                result_dict['EID'] = retry_find_element(each_employee, By.CSS_SELECTOR, 'span.jssEEFormsRewrite11').text.replace('(','').replace(')','')
                
                result_dict['position'] = retry_find_element(each_employee, By.CSS_SELECTOR, 'div.css-eix830').text
                
                

                time.sleep(5)
                
                phones_emails_managers = retry_find_elements(each_employee, By.CSS_SELECTOR, 'div.css-1uf4nsy')
                driver.implicitly_wait(5)

                phones_emails_managers[0].find_elements(By.CSS_SELECTOR, 'button')
                driver.implicitly_wait(5)
                print("\tfound buttons")
                for index_no in range(3):
                    phones_emails_managers[index_no].click()
                    driver.implicitly_wait(10)

                    lines = retry_find_element(driver, By.CSS_SELECTOR, 'div.css-djbxmn').text.splitlines()
                    driver.implicitly_wait(5)

                    key, key_prefix, value = "", "", ""
                    if index_no == 0:
                        key_prefix = "Phone Number"
                    if index_no == 1:
                        key_prefix = "Email"
                    if index_no == 2:
                        key_prefix = "Manager"
                    cnt = 0
                    for each in lines:
                        if index_no < 2:
                            if cnt % 2 == 0:
                                key = key_prefix + " - " + each
                            if cnt % 2 == 1:
                                value = each
                                result_dict[key] = value
                        else:
                            if cnt % 4 == 0:
                                key = key_prefix + " - " + each
                            if cnt % 4 == 2:
                                value = each
                                result_dict[key] = util.name_formatter(value)
                        cnt = cnt + 1
                element = retry_find_element(driver, By.CSS_SELECTOR, 'div.css-1kf24an')
                left_tab_components = retry_find_elements(element, By.CSS_SELECTOR, 'div.css-j486mh')
                for each in left_tab_components:
                    lines = each.text.splitlines()
                    key, value = "", ""
                    cnt = 0
                    for line in lines:
                        if cnt%4 == 0:
                            key = line
                        if cnt%4 == 1:
                            value = line
                        cnt = cnt + 1
                    result_dict[key] = util.name_formatter(value)

                employment_table = retry_find_element(driver, By.CSS_SELECTOR, 'div.css-18hyvxw')
                contents = retry_find_elements(employment_table, By.CSS_SELECTOR, 'div.css-j486mh')
                for each in contents:
                    str_lines = each.text.splitlines()
                    result_dict[str_lines[0]] = str_lines[1]

                result.append(result_dict)
                
                print(employee_id)
                employee_ids.append(employee_id)

                ###

        driver.execute_script("window.scrollBy(0, " + str(one_row_height * 10) + ")","")
        current_page_y_offset = driver.execute_script("return window.pageYOffset;")
        if current_page_y_offset == before_page_y_offset:
            not_hit_bottom = 0
        else:
            before_page_y_offset = current_page_y_offset
    
    except StaleElementReferenceException:

        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        
        # print(traceback.format_exc())
        print("StaleElementReferenceException: Reopening the page...")
        driver.get("https://www.paycomonline.net/v4/cl/web.php/employee/changes")
        delay = 10
        WebDriverWait(driver, delay).until(
            expected_conditions.presence_of_element_located((By.CLASS_NAME, 'css-vkr5jj')))
        print("'Make Employee Changes' page loaded")
   




	found buttons
(0751)
	found buttons
(1508)
	found buttons
(0764)
	found buttons
(0841)
	found buttons
(0623)
	found buttons
(0731)
	found buttons
(1038)
	found buttons
(1149)
	found buttons
(1663)
	found buttons
(1424)
	found buttons
(0212)
	found buttons
(1088)
	found buttons
(1083)
	found buttons
(1354)
	found buttons
(0605)
	found buttons
(1629)
	found buttons
(1691)
	found buttons
(1665)
	found buttons
(1346)
	found buttons
(1646)
	found buttons
(1582)
	found buttons
(0722)
	found buttons
(1631)
	found buttons
(0972)
	found buttons
(1718)
	found buttons
(1689)
	found buttons
(0497)
	found buttons
(0723)
	found buttons
(1656)
	found buttons
(1676)
	found buttons
(1617)
	found buttons
(1405)
	found buttons
(1222)
	found buttons
(1139)
	found buttons
(1403)
	found buttons
(1150)
	found buttons
(1594)
	found buttons
(1690)
	found buttons
(1687)
<class 'selenium.common.exceptions.StaleElementReferenceException'> 245801455.py 73
StaleElementReferenceException: Reopening the page...
'Mak

KeyboardInterrupt: 